In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
X_train, y_train = pd.read_pickle('./data/pickle/preproc/df_patient_admit_icu__20210130_TRAIN_final.pkl')

## URGENCY SCORE

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from customTransformers import ColumnSelectTransformer, DiagnosisFrameTransformer, EstimatorTransformer, LinearNonlinear
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

### Logistic on base cagetoricals + age, RF on diagnoses, xgboost on ensemble

In [ ]:
## TRYING TO IMPLEMENT 4 OVR CLASSIFICATION SYSTEMS FOR ICU_URGENCY.
# 1. CREATE AND TUNE OVR for demographics
# 2. use multinomial classifier for diagnoses
# 3. use all 5 outputs as features in a second multinomial to pool results
from sklearn.preprocessing import OrdinalEncoder

cols = ['GENDER','ADMISSION_TYPE','ADMISSION_LOCATION','INSURANCE','LANGUAGE',
       'RELIGION','MARITAL_STATUS','ETHNICITY']

num_cols = ['ADMIT_AGE']
demog_feats = FeatureUnion([
    ('stdscl', ColumnTransformer([('numerical', StandardScaler(), num_cols)])),
    ('ohe',  ColumnTransformer([('categorical', OneHotEncoder(handle_unknown='ignore'), cols)]))
])

demog_pipe = Pipeline([
    ('features', demog_feats),
    ('reg', LogisticRegression(solver='saga'))
])
demog_params = {
    'reg__penalty': ['l1','l2','none'],
    'reg__C': [0.01, 0.1, 1, 10],
    'reg__class_weight': ['none','balanced'],
    'reg__multi_class': ['ovr','multinomial'],
}

ordenc = OrdinalEncoder(categories=[['stable','questionable','urgent','immediate']])
y_train_enc = ordenc.fit_transform(y_train.to_numpy().reshape(-1,1))

lin_gs_classifier = GridSearchCV(demog_pipe, demog_params, cv=StratifiedKFold(n_splits=5, shuffle=True),verbose=3,n_jobs=-1)
lin_est = lin_gs_classifier.fit(X_train,y_train_enc)

lin_est.best_params_,lin_est.best_score_

In [ ]:
diagn_pipe = Pipeline([
    ('cst', ColumnSelectTransformer(['DIAGNOSIS'])),
    ('dst', DiagnosisFrameTransformer()),
    ('count',CountVectorizer(max_features=3000)),
    ('tfid', TfidfTransformer()),
    ('rfc', RandomForestClassifier())
])

diagn_params = {
    'rfc__criterion': ['gini','entropy'],
    'rfc__max_features': ['auto', 'sqrt', 'log2'],
    'rfc__min_samples_split': [2,5,8,11],
    'rfc__min_samples_leaf': [1,4,7],
    'rfc__class_weight': ['balanced', 'balanced_subsample'],
}

diagn_gs_classifier = GridSearchCV(diagn_pipe, diagn_params, cv=StratifiedKFold(n_splits=5, shuffle=True),verbose=3,n_jobs=4)
tfidf_est = diagn_gs_classifier.fit(X_train,y_train_enc)
tfidf_est.best_params_,tfidf_est.best_score_

In [ ]:
demog_est = LogisticRegression(solver='saga', C=0.1, class_weight='none', 
                               penalty='l2', multi_class='ovr')
demog_pipe = Pipeline([
    ('features', demog_feats),
    ('reg', demog_est)
])

diagn_est = RandomForestClassifier(class_weight='balanced',criterion='gini',
                                  max_features='sqrt',min_samples_leaf=1,
                                  min_samples_split=2)
diagn_pipe = Pipeline([
    ('cst', ColumnSelectTransformer(['DIAGNOSIS'])),
    ('dst', DiagnosisFrameTransformer()),
    ('count',CountVectorizer(max_features=3000)),
    ('tfid', TfidfTransformer()),
    ('rfc', diagn_est)
])

union = FeatureUnion([
    ('demog', EstimatorTransformer(demog_pipe)),# FeatureUnions use the same syntax as Pipelines
    ('diagn',EstimatorTransformer(diagn_pipe))
])


full_pipe = Pipeline([
    ('union',union),
    ('xgb', GradientBoostingClassifier())
])

xgb_params = {
    'xgb__loss': ['deviance','exponential'],
    'xgb__learning_rate': [0.05, 0.1, 0.2],
    'xgb__subsample': [0.8, 1], # max is 1
    'xgb__criterion': ['friedman_mse'],
    'xgb__min_samples_leaf': [1, 4, 7],
    'xgb__max_depth': [2,3,4],
    'xgb__max_features': ['auto']
}

xgb_gs_classifier = GridSearchCV(full_pipe, xgb_params, cv=StratifiedKFold(n_splits=3, shuffle=True),verbose=3,n_jobs=4)
xgb_est = xgb_gs_classifier.fit(X_train,y_train_enc)

xgb_est.best_params_,xgb_est.best_score_

### Logistic on all features

In [4]:
cols = ['GENDER','ADMISSION_TYPE','ADMISSION_LOCATION','INSURANCE','LANGUAGE',
       'RELIGION','MARITAL_STATUS','ETHNICITY']
num_cols = ['ADMIT_AGE']

ohe = ColumnTransformer([('categorical', OneHotEncoder(handle_unknown='ignore'), cols)])
demog_feats = FeatureUnion([
    ('stdscl', ColumnTransformer([('numerical', StandardScaler(), num_cols)])),
    ('ohe',  ohe)
])

count = CountVectorizer(max_features=3000)
diagn_feats = Pipeline([
    ('cst', ColumnSelectTransformer(['DIAGNOSIS'])),
    ('dst', DiagnosisFrameTransformer()),
    ('count', count),
    ('tfid', TfidfTransformer())
])

feats_union = FeatureUnion([
    ('demog_feats', demog_feats),
    ('diagn_feats', diagn_feats)
])

lin_pipe = Pipeline([
    ('features', feats_union),
    ('reg', LogisticRegression(solver='saga'))
])

lin_params = {
    'reg__penalty': ['l1','l2'],
    'reg__C': [1, 5, 10],
    'reg__class_weight': ['none','balanced'],
    'reg__multi_class': ['ovr','multinomial'],
}

ordenc = OrdinalEncoder(categories=[['stable','questionable','urgent','immediate']])
y_train_enc = ordenc.fit_transform(y_train.to_numpy().reshape(-1,1))
lin_gs_classifier = GridSearchCV(lin_pipe, lin_params, cv=StratifiedKFold(n_splits=5, shuffle=True),verbose=3,n_jobs=-1)
lin_est = lin_gs_classifier.fit(X_train,y_train_enc)

lin_est.best_params_,lin_est.best_score_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  8.2min finished
/Users/adamgifford/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/adamgifford/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/adamgifford/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/adamgifford/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter 

({'reg__C': 1,
  'reg__class_weight': 'none',
  'reg__multi_class': 'ovr',
  'reg__penalty': 'l2'},
 0.6445177912568465)

In [5]:
file = './data/pickle/models/log__URGENCY__20210130.pkl'

ohe.fit(X_train)
feats_union.fit(X_train)
diagn_feats.fit(X_train)
model_data = {
    'numeric_cols': num_cols,
    'categorical_cols': cols,
    'diagnosis_col': ['DIAGNOSIS'],
    'ohe_categoricals': ohe,
    'feature_union': feats_union,
    'diagn_vect': diagn_feats,
    'ord_enc_y': ordenc,
    'gridsearch': lin_gs_classifier,
    'estimator': lin_est
}
pickle.dump(model_data,open(file,'wb'))

In [6]:
file = './data/pickle/models/log__URGENCY__20210130.pkl'
model_data = pickle.load(open(file,'rb'))
X_test, y_test = pd.read_pickle('./data/pickle/preproc/df_patient_admit_icu__20210126_TEST_final.pkl')

In [7]:
est = model_data['estimator']
oh = model_data['ohe_categoricals']
dv = model_data['diagn_vect']
fu = model_data['feature_union']
cats = model_data['categorical_cols']
# del model_data
# est

In [8]:
len(oh.get_feature_names())

125

In [9]:
len(dv.named_steps.count.get_feature_names())

3000

In [10]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral4
output_notebook()

Loading BokehJS ...

testing expected functionality for web app input

In [11]:
foo = pd.DataFrame(X_test.iloc[0]).T
foo = foo.to_dict()
bar = {key: val for key in foo for val in foo[key].values() }
xx_test = pd.DataFrame(bar,index=[0])
xx_test

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,ADMIT_AGE,ICUSTAY_ID,DBSOURCE,INTIME,LOS,DAYS_ADM_TO_ICU,SAMEDAY_ADM_TO_ICU,ADM_TO_ICU_100p,ADM_TO_ICU_90m,ICU_URGENCY
0,15896,F,124497,2141-05-25 23:16:00,2141-07-06 17:25:00,NEWBORN,CLINIC REFERRAL/PREMATURE,Private,***,CATHOLIC,...,0,276327,carevue,2141-05-25 23:25:40,41.7733,0.006713,1,49,0,immediate


In [12]:
y_pred = int(model_data['estimator'].predict(xx_test)[0])

In [13]:
y_pred_proba =  model_data['estimator'].predict_proba(xx_test)[0]

In [14]:
def make_urgency_plot(y_pred_proba):
    factors = ['stable','questionable','urgent','immediate']
    source = ColumnDataSource(data=dict(factors=factors, probs=y_pred_proba, color=Spectral4))

    p = figure(x_range=factors, plot_height=250, title="IC-U Risk Factor",
            plot_width=300, toolbar_location=None, tools="")

    p.vbar(x='factors', top='probs', color='color', width=0.4, source=source)

    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.yaxis.axis_label = 'P(category)'

    return p

In [15]:
p = make_urgency_plot(y_pred_proba)
show(p)

In [16]:
def getFeatureNames(categorical,bow,numerical=['ADMIT_AGE']):
    feature_names = numerical
    feature_names.extend([c.split('_')[-1] for c in categorical.get_feature_names()])
    feature_names.extend(bow.get_feature_names())
    
    return feature_names

In [17]:
def createFeatureCoeffDict(coefs,features):
    feats_coeffs = [(f, c) for c,f in zip(coefs,features)]
    return feats_coeffs

In [29]:
def getMostImportantFeaturesUrg(feature_imp_urg, x_test, n_most=4):
    x_test = fu.transform(x_test).toarray()[0]
    features_scores = [(f[0], x * f[1]) for x,f in zip(x_test,feature_imp_urg)]
    most_important = sorted(features_scores, key=lambda x: abs(x[1]))
    return most_important[-n_most:]

In [19]:
feature_names_urg = getFeatureNames(ohe, dv.named_steps.count,
                                    num_cols)

In [20]:
feature_imp_urg = createFeatureCoeffDict(est.best_estimator_.named_steps.reg.coef_[y_pred],
                                        feature_names_urg)

In [21]:
xx_test

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,INSURANCE,LANGUAGE,RELIGION,...,ADMIT_AGE,ICUSTAY_ID,DBSOURCE,INTIME,LOS,DAYS_ADM_TO_ICU,SAMEDAY_ADM_TO_ICU,ADM_TO_ICU_100p,ADM_TO_ICU_90m,ICU_URGENCY
0,15896,F,124497,2141-05-25 23:16:00,2141-07-06 17:25:00,NEWBORN,CLINIC REFERRAL/PREMATURE,Private,***,CATHOLIC,...,0,276327,carevue,2141-05-25 23:25:40,41.7733,0.006713,1,49,0,immediate


In [22]:
foo=fu.transform(xx_test)

In [30]:
most_imp_feats_urg = getMostImportantFeaturesUrg(feature_imp_urg, xx_test)

In [31]:
most_imp_feats_urg

[('CLINIC REFERRAL/PREMATURE', 0.2452936432802994),
 ('UNKNOWN (DEFAULT)', 0.298331752872842),
 ('newborn', -0.5147241038765541),
 ('NEWBORN', 1.6389059014202116)]

In [35]:
def make_urgency_factors_plot(most_imp_feats_urg):
    y = [m[0] for m in most_imp_feats_urg]
    x = [abs(m[1]) for m in most_imp_feats_urg]
    x = [val/np.round(max(x)) for val in x]

    source = ColumnDataSource(dict(factors=y, scores=x,))

    p = figure(y_range=y, plot_height=250, title="Main Factors",
            plot_width=400, toolbar_location=None, tools="")

    p.hbar(y='factors', right='scores', height=0.4, source=source)

    p.ygrid.grid_line_color = None
    p.x_range.start = 0
    p.xaxis.visible = False
    return p

In [36]:
p_urg_factors = make_urgency_factors_plot(most_imp_feats_urg)

In [37]:
show(p_urg_factors)

In [62]:
est.best_estimator_.named_steps.reg.coef_

array([[ 0.06388616, -1.40906547, -1.31259421, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.19920691, -0.91501042, -0.90308329, ...,  0.        ,
         0.        ,  0.53267716],
       [-0.04237232, -1.31372519, -1.37530598, ...,  0.        ,
         0.        ,  0.        ],
       [-0.08885758,  0.01463244,  0.        , ...,  0.        ,
         0.        , -0.49348576]])

In [51]:
bam = pd.DataFrame(baz,index=[0])

In [60]:
est.predict(xx_test)

array([3.])

In [61]:
est.best_estimator_.named_steps.reg.coef_

array([[ 0.06388147, -1.37698274, -1.28048517, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.19920232, -0.98095228, -0.96901725, ...,  0.        ,
         0.        ,  0.53344316],
       [-0.04236483, -1.30158279, -1.36317284, ...,  0.        ,
         0.        ,  0.        ],
       [-0.08886696,  0.01463558,  0.        , ...,  0.        ,
         0.        , -0.49353617]])